In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../')

In [4]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier
from BackGround_modules.Class_2_Classify_Fourier_Img import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

The default supplementary data a Zero img, change accordingly when classify other years imgs



### Step_0_Define_Basic_Parameters

In [5]:
# define the year range
year_start = [f'{i}-01-01' for i in range(1987,2020,3)]
year_end   = [f'{i}-12-31' for i in range(1989,2020,3)]

year_range = list(zip(year_start,year_end))

# import some spatial constrains
North_china_plain_boundary = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [6]:
year_range

[('1987-01-01', '1989-12-31'),
 ('1990-01-01', '1992-12-31'),
 ('1993-01-01', '1995-12-31'),
 ('1996-01-01', '1998-12-31'),
 ('1999-01-01', '2001-12-31'),
 ('2002-01-01', '2004-12-31'),
 ('2005-01-01', '2007-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2014-01-01', '2016-12-31'),
 ('2017-01-01', '2019-12-31')]

### Step_1_Create_the_Fourier_Img

In [7]:
# reverse the year_range order so that the most recent will be exported first    
for span in year_range[::-1]:
    
    start = span[0]
    end   = span[1]
    
    split = '-'
    span_name = f'{start.split(split)[0]}_{end.split(split)[0]}'

    # define the asset path for exporting
    asset_path = 'users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3'
    # define the name for exporting
    export_name = f'Fourier_img_{span_name}_harmonic_3'


    # make the Fourier img, the default is to make time series
    # of ['NDVI', 'NDBI', 'EVI']
    Fourier_making = Make_Fourier( start_date = start,
                                   end_date   = end,
                                   harmonics  = 3 )
    Fourier_making.Stp_1_Create_hamonic_names()
    Fourier_making.Stp_2_Add_harmonics()
    Fourier_making.Stp_3_Harmonic_fit()

    # Get the Fourier img
    Fourier_img = Fourier_making.harmonicTrendCoefficients

    # export the Fourier img
#     task = ee.batch.Export.image.toAsset( 
#                                                 image          = Fourier_img,
#                                                 description    = f'Exporting {export_name}',
#                                                 assetId        = f'{asset_path}/{export_name}',
#                                                 region         = North_china_plain_boundary.geometry().bounds(),
#                                                 scale          = 30,
#                                                 maxPixels      = int(1e13))
#     task.start()

    # print out the process
    print(f'Exporting {asset_path}/{export_name}')

Analyzing the images of 2002_2004
Exporting users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_2002_2004_harmonic_3
Analyzing the images of 1999_2001
Exporting users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_1999_2001_harmonic_3
Analyzing the images of 1996_1998
Exporting users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_1996_1998_harmonic_3
Analyzing the images of 1993_1995
Exporting users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_1993_1995_harmonic_3
Analyzing the images of 1990_1992
Exporting users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_1990_1992_harmonic_3
Analyzing the images of 1987_1989
Exporting users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_1987_1989_harmonic_3


### Step_2_Create_Cloud_Free_Img

In [15]:
for span in year_range[::-1]:
    
    # find the end year to determine the 
    start = span[0]
    end   = span[1]
    
    split = '-'
    span_name = f'{start.split(split)[0]}_{end.split(split)[0]}'

    # define the asset path for exporting
    asset_path = 'users/wang8052664/Cloud_Free_Img'
    # define the name for exporting
    export_name = f'Landsat_cloud_free_{span_name}'
    
    
    # find the right Landsat RAW data set
    if int(end.split(split)[0]) <= 2010:
        Landsat_img = ee.ImageCollection("LANDSAT/LT05/C01/T1")
        
    elif int(end.split(split)[0]) <= 2013:
        Landsat_img = ee.ImageCollection("LANDSAT/LE07/C01/T1")
        
    else:
        Landsat_img = ee.ImageCollection("LANDSAT/LC08/C01/T1")
        
    
    # create the cloud-free img using SimpleComposite
    Filtered_img = Landsat_img.filterBounds(North_china_plain_boundary)\
                              .filterDate(start,end)
    
    cloud_free = ee.Algorithms.Landsat.simpleComposite(Filtered_img)\
                              .clip(North_china_plain_boundary)
        
    # export
    task = ee.batch.Export.image.toAsset(   image       = cloud_free,
                                            description = f'Exporting {export_name}',
                                            assetId     = f'{asset_path}/{export_name}',
                                            region      = North_china_plain_boundary.geometry(),
                                            scale       = 30,
                                            maxPixels   = int(1e13))
    task.start()
    
    # print out the process
    print(f'Exporting {export_name}')

Exporting Landsat_cloud_free_2017_2019
Exporting Landsat_cloud_free_2014_2016
Exporting Landsat_cloud_free_2011_2013
Exporting Landsat_cloud_free_2008_2010
Exporting Landsat_cloud_free_2005_2007
Exporting Landsat_cloud_free_2002_2004
Exporting Landsat_cloud_free_1999_2001
Exporting Landsat_cloud_free_1996_1998
Exporting Landsat_cloud_free_1993_1995
Exporting Landsat_cloud_free_1990_1992
Exporting Landsat_cloud_free_1987_1989
